In [4]:
# === Build organized_highres.csv from 64 m stacks (assumes consistent band order) ===
import os, re, numpy as np, pandas as pd, geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from tqdm import tqdm

# ---------------- CONFIG ----------------
GPKG_PATH   = "gpk_spatial_grid.gpkg"           # your grid file (layer 'gpk_spatial_grid')
GPKG_LAYER  = "gpk_spatial_grid"
LABELS_CSV  = "Dataset_WFI.csv"                 # must have id, 20_labels, 21_labels, 22_labels
STACKS_DIR  = "Stacks_RGBNir_BAI_EVI_GEMI_NDVI_NDWI"  # folder with .tif/.tiff stacks
OUT_CSV     = "organized_highres.csv"

# Canonical column order for output
CANON = ["Blue","Green","Red","NIR","BAI","EVI","GEMI","NDVI","NDWI"]

# Band order inside each TIFF (1-based → 0-based below)
# Folder name indicates: [R, G, B, NIR, BAI, EVI, GEMI, NDVI, NDWI]
TIFF_ORDER = {
    "Red":   0,  # band 1
    "Green": 1,  # band 2
    "Blue":  2,  # band 3
    "NIR":   3,  # band 4
    "BAI":   4,  # band 5
    "EVI":   5,  # band 6
    "GEMI":  6,  # band 7
    "NDVI":  7,  # band 8
    "NDWI":  8,  # band 9
}

# Export order
EXPORT_IDX = {
    "Blue":  TIFF_ORDER["Blue"],
    "Green": TIFF_ORDER["Green"],
    "Red":   TIFF_ORDER["Red"],
    "NIR":   TIFF_ORDER["NIR"],
    "BAI":   TIFF_ORDER["BAI"],
    "EVI":   TIFF_ORDER["EVI"],
    "GEMI":  TIFF_ORDER["GEMI"],
    "NDVI":  TIFF_ORDER["NDVI"],
    "NDWI":  TIFF_ORDER["NDWI"],
}

# Extract date from filename (e.g. ...20200131...tif)
DATE_RE = re.compile(r"(\d{8})")
# ----------------------------------------


# --- 1) Load grid and labels ---
print("Reading grid…")
grid = gpd.read_file(GPKG_PATH, layer=GPKG_LAYER)
if grid.crs is None:
    grid = grid.set_crs(4326)
elif grid.crs.to_epsg() != 4326:
    grid = grid.to_crs(4326)

grid = grid[["id","geometry"]].copy()

print("Reading labels…")
labels = pd.read_csv(LABELS_CSV, low_memory=False)
keep = [c for c in ["id","20_labels","21_labels","22_labels"] if c in labels.columns]
labels = labels[keep].drop_duplicates("id")

grid = grid.merge(labels, on="id", how="left")
grid["22_labels"] = grid["22_labels"].astype(str)
grid = grid[grid["22_labels"].isin(["nb","tb"])].reset_index(drop=True)
print("Samples after dropping 'pb':", len(grid))
assert len(grid) > 0, "No samples retained!"

# Centroids for sampling
cent = grid.geometry.centroid
pts_xy = np.array([(p.x, p.y) for p in cent], dtype="float64")
N = len(grid)

# --- 2) Find TIFFs and extract dates ---
tif_paths = [os.path.join(STACKS_DIR, f) for f in os.listdir(STACKS_DIR)
             if f.lower().endswith((".tif",".tiff"))]

def get_date(fn):
    m = DATE_RE.search(os.path.basename(fn))
    return m.group(1) if m else None

pairs = [(p, get_date(p)) for p in tif_paths]
pairs = [(p,d) for (p,d) in pairs if d is not None]
pairs.sort(key=lambda x: x[1])
assert pairs, "No valid TIFFs found (date not detected). Check STACKS_DIR or DATE_RE."
print(f"Found {len(pairs)} stacks. First 3:", [os.path.basename(p) for p,_ in pairs[:3]])

# --- 3) Sampling loop ---
band_series = {b: [] for b in CANON}
all_dates = []

for tif_path, dstr in tqdm(pairs, desc="Sampling", ncols=100):
    all_dates.append(dstr)
    with rasterio.open(tif_path) as src:
        # ensure in EPSG:4326
        if (src.crs is None) or (src.crs.to_epsg() != 4326):
            with WarpedVRT(src, crs="EPSG:4326", resampling=Resampling.nearest) as vrt:
                vals = np.array(list(vrt.sample(pts_xy)))  # (N,9)
        else:
            vals = np.array(list(src.sample(pts_xy)))      # (N,9)

    # append bands in canonical order
    for name in CANON:
        band_series[name].append(vals[:, EXPORT_IDX[name]])

# --- 4) Convert to per-sample lists ---
T = len(all_dates)
print(f"Assembled N={N} samples across T={T} dates.")

per_sample = {}
for name in CANON:
    arr = np.stack(band_series[name], axis=0).T   # (N,T)
    per_sample[name] = [row.tolist() for row in arr]

# --- 5) Build output table ---
out = pd.DataFrame({
    "id": grid["id"].values,
    "20_labels": grid["20_labels"].values if "20_labels" in grid.columns else None,
    "21_labels": grid["21_labels"].values if "21_labels" in grid.columns else None,
    "22_labels": grid["22_labels"].values
})
for name in CANON:
    out[name] = per_sample[name]
out["Dates"] = [all_dates] * N

print("Preview:\n", out.head(1))

# --- 6) Save CSV ---
print("Saving CSV…")
out.to_csv(OUT_CSV, index=False)
print("✅ Done. Wrote:", OUT_CSV)


Reading grid…
Reading labels…
Samples after dropping 'pb': 35811


/tmp/ipykernel_22147/3141529247.py:73: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent = grid.geometry.centroid


Found 235 stacks. First 3: ['20200110CBERS4A_WFI20613220200110.tif.tif', '20200115CBERS4A_WFI20513220200115.tif.tif', '20200120CBERS4A_WFI20413220200120.tif.tif']


Sampling: 100%|███████████████████████████████████████████████████| 235/235 [10:28<00:00,  2.68s/it]


Assembled N=35811 samples across T=235 dates.
Preview:
    id 20_labels 21_labels 22_labels  \
0   1        nb        nb        nb   

                                                Blue  \
0  [0.05700000002980232, 0.03500000014901161, 0.0...   

                                               Green  \
0  [0.054999999701976776, 0.039000000804662704, 0...   

                                                 Red  \
0  [0.0430000014603138, 0.027000000700354576, 0.0...   

                                                 NIR  \
0  [0.09300000220537186, 0.08399999886751175, 0.0...   

                                                 BAI  \
0  [2.2880001068115234, 1.7109999656677246, 2.272...   

                                                 EVI  \
0  [0.13600000739097595, 0.14399999380111694, 0.1...   

                                                GEMI  \
0  [0.5139999985694885, 0.5509999990463257, 0.460...   

                                                NDVI  \
0  [0.256000012159